# STARTING TWO CELLS IMPORT LIBRARIES AND DEFINE FUNCTIONS

In [ ]:
import numpy as np
import pandas as pd
import ligo.skymap.io
import healpy as hp
import matplotlib.pyplot as plt

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np

import ligo.skymap.io
import healpy as hp
import matplotlib.pyplot as plt

skym = ligo.skymap.io.fits.read_sky_map('farah/allsky/36.fits', distances=True)
skymap = hp.ud_grade(skym[0][0], 512)
print('Yes')


def get_probabilities(skymap, ra, dec, radius=0.35*u.deg):
    """
    Compute the probabilities covered in a grid of ra, dec with radius
    given a healpix skymap

    Pass the legal RA and Dec lists to this function

    Note: this radius is such that for the default sky grid (nside=128),
    fields overlap such that the area of the sky becomes 1.83 * 4pi steradians.
    As a result, the sum of probabilities will be 1.83
    """
    # fact : int, optional
    # Only used when inclusive=True. The overlapping test will be done at
    # the resolution fact*nside. For NESTED ordering, fact must be a power of 2, less than 2**30,
    # else it can be any positive integer. Default: 4.
    fact = 1
    nside_skymap = hp.npix2nside(len(skymap))
    tile_area = np.pi * radius.to(u.deg).value ** 2
    pixel_area = hp.nside2pixarea(nside_skymap, degrees=True)
    probabilities = np.zeros(len(ra))
    vecs = hp.ang2vec(ra.to(u.deg).value, dec.to(u.deg).value, lonlat=True)
    for i in range(len(ra)):
        sel_pix = hp.query_disc(nside_skymap, vecs[i], radius.to(
            u.rad).value, inclusive=True, fact=fact)
        probabilities[i] = np.sum(skymap[sel_pix]) * \
            tile_area / pixel_area / len(sel_pix)
        probabilities[i] = np.sum(skymap[sel_pix])/ len(sel_pix)
    return probabilities



def get_top_tiles(probabilities, frac=0.99):
    """
    probabilities may not add up to 1
    return indices of tiles that add up to frac of total
    """
    sortorder = np.argsort(probabilities)
    p_cum = np.cumsum(probabilities[sortorder]) / np.sum(probabilities)
    startind = np.where(p_cum > 1 - frac)[0][0]
    top_tiles = sortorder[startind:]
    return np.flip(top_tiles)




tiles = pd.read_csv('tiles_GIT_7.csv')
ralist, declist = np.array(tiles['RA_Center'])*u.deg, np.array(tiles['DEC_Center'])*u.deg

probs = get_probabilities(skymap, np.array([108.756])*u.deg, np.array([22.9])*u.deg)



probs = get_probabilities(skymap, ralist, declist)
tt = get_top_tiles(probs)
len(tt)


# THIS CELL CALCULATES MEAN DISTANCE AND AREA FOR A GIVEN MAP. I DID THIS MANUALLY TOWARDS THE END AS SOME MAPS CAUSED PYTHON TO CRASH

## SEE COMMENT AFTER FUNCTION DEFINITION - THAT IS WHERE MAP NUMBER IS ENTERED


In [ ]:
nlist = np.load('ns.npy')
nlist
#df = pd.read_csv('farah/injections.dat', skiprows=0, delimiter="\t")
df = pd.read_csv('farah/injections.csv')
np.argmin(df.distance)

In [ ]:

def get_top_tiles(probabilities, frac=0.99):
    """
    probabilities may not add up to 1
    return indices of tiles that add up to frac of total
    """
    sortorder = np.argsort(probabilities)
    p_cum = np.cumsum(probabilities[sortorder]) / np.sum(probabilities)
    startind = np.where(p_cum > 1 - frac)[0][0]
    top_tiles = sortorder[startind:]
    return np.flip(top_tiles)



#####------------------THE LINE BELOW TELLS MAP NUMBER TO DO CALCULATIONS FOR, HERE NSBH_1500-----------------#####

badind = [11, 12, 16, 20, 49, 51, 63, 69, 85, 86,
         104, 107, 112, 118, 119, 130, 134,138, 152, 167,171, 180,
         214,216,222,230,236,262,280,288,296,
         313, 314, 325, 339, 341, 363, 377, 380, 383,
         412, 422, 446, 470, 483,
         508, 521, 522, 534, 547, 570, 579, 581, 587, 592,
         623, 647, 652, 686, 687,
         708, 717, 723, 740, 760, 766, 798,
         804, 808, 829, 847, 864, 873, 896, 897, 899,
         908, 914, 929, 948, 956, 973,
         1003, 1004, 1009, 1024, 1034, 1037, 1056, 1076, 1095,
         1101, 1111, 1127]

ind = 900
n = nlist[ind]
n = 900
#n = 405
#n = 3535
skym = ligo.skymap.io.fits.read_sky_map('farah/allsky/{}.fits'.format(n), distances=True)
plt.figure(figsize=(12,10))
hp.mollview(skym[0][1])
hp.mollview(skym[0][0])
tt = get_top_tiles(skym[0][0])
li = len(tt)
tt = tt[~np.isinf(skym[0][1][tt])]
#print('Distance =',np.sum(skym[0][0][tt]*skym[0][1][tt])/np.sum(skym[0][0][tt]))
print("Distance =",df['distance'][np.where(df.simulation_id==n)[0]])
skymap = hp.ud_grade(skym[0][0], 512)
probs = get_probabilities(skymap, ralist, declist)
tt = get_top_tiles(probs, 0.99)
print('Num GIT tiles = ',len(tt))
tt = get_top_tiles(skymap, 0.99)
a = len(tt)*hp.nside2pixarea(512)*3282.8
print('Sq deg = ',a)
print('Mass2 = ', df.mass2[n])
print('n = ',n)

####  15, 48, 87, 25

In [ ]:
np.where(df.simulation_id==n)

# AREA CALCULATION END

In [ ]:
skym = ligo.skymap.io.fits.read_sky_map('bns_astro/allsky/47.fits', distances=True)
skymap = skym[0][0]

np.sqrt(len(skymap)/12)

# FETCH CSV FILES FOR GALAXY CATALOG AND MISSING MASS - DIRECTORY STRUCTURE IS DIFF ON GITHUB, PLS CHANGE ACCORDINGLY!

In [ ]:
cat2 = pd.read_csv('../../new_reduced_galcat.csv')
new = pd.read_csv('../../missing_mass.csv')
cat2['Dist'] = cat2['DistMpc']

In [ ]:
d1 = pd.read_csv('full.csv')

names = d1['num'][304:]

for name in names:
    
    skym = ligo.skymap.io.fits.read_sky_map('farah/allsky/{}.fits'.format(name), distances=True)
    skymap = hp.ud_grade(skym[0][0], 512)
    
    #print('\n\n\n',name,'\n\n\n')

    tiles = pd.read_csv('tiles_GIT_7.csv')
    ralist, declist = np.array(tiles['RA_Center'])*u.deg, np.array(tiles['DEC_Center'])*u.deg
    probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(probs, 0.99)
    print(name,len(tt))

# MAIN CELL START

## AFTER FINDING GOOD LOCALIZATION MAPS, I USED TO LIST THEM IN NSBH_FULL.CSV AND RUN THE MASS-FILLING ON THOSE MAPS ONLY. 

## LOOP STATEMENT CAN BE MODIFIED AND EARLIER CELL USED TO INCLUDE AREA CALCULATION + IF STATEMENT TO RUN THE MASS FILLING IF SMALL AREA - then manually checking maps not needed


## Please make sure to change 'NSBH' to 'BNS' at a couple of places inside this cell when running for bns. 

In [ ]:
d1 = pd.read_csv('full.csv')

names = d1['num'][0:]

for name in names:
    
    skym = ligo.skymap.io.fits.read_sky_map('farah/allsky/{}.fits'.format(name), distances=True)
    skymap = hp.ud_grade(skym[0][0], 512)
    
    print('\n\n\n##############################',name,'#################################\n\n\n')

    tiles = pd.read_csv('tiles_GIT_7.csv')
    ralist, declist = np.array(tiles['RA_Center'])*u.deg, np.array(tiles['DEC_Center'])*u.deg
    probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(probs, 0.99)
    len(tt)

    distmu = hp.ud_grade(skym[0][1], 512)
    distsigma = hp.ud_grade(skym[0][2], 512)
    prob2d = hp.ud_grade(skym[0][0], 512)


    radius = 0.35*u.deg
    fact = 1
    nside_skymap = 512

    tile_area = np.pi * radius.to(u.deg).value ** 2
    pixel_area = hp.nside2pixarea(nside_skymap, degrees=True)

    sigma_m = np.zeros(len(tt))

    mpcd = np.arange(0,1001,20)


    
    sigma_m_empty_nodist = np.zeros(len(tt))
    sigma_m_only = np.zeros(len(tt))
    
    #if len(tt)

    for h in range(len(tt)):
        ra = ralist[tt[h]]
        dec = declist[tt[h]]

        vecs = hp.ang2vec(ra.to(u.deg).value, dec.to(u.deg).value, lonlat=True)

        sel_pix = hp.query_disc(nside_skymap, vecs, radius.to(u.rad).value, inclusive=True, fact=fact)
        
        m,s = np.median(distmu[sel_pix]), np.median(distsigma[sel_pix])
        
        currcat = cat2[np.isin(np.array(cat2.PIX_ID_512), sel_pix)][['Dist','Mstar']]
        currcat = currcat[(currcat.Dist>=m-3*s) & (currcat.Dist<=m+3*s)]
        
        summass = np.sum(currcat['Mstar'])
        masses3 = summass * np.sum(prob2d[sel_pix])
        sigma_m_only[h] = summass
        sigma_m_empty_nodist[h] = masses3
        
        prob2d[sel_pix] = 0.0
        print(h)
        
    df = pd.DataFrame({'Unfilled2D':sigma_m_empty_nodist,'MassOnly_Distcut':sigma_m_only})
    df.to_csv('Orderings-2D-GIT/Map_{}.csv'.format(name))

In [ ]:
   for h in range(len(tt)):
        ra = ralist[tt[h]]
        dec = declist[tt[h]]

        vecs = hp.ang2vec(ra.to(u.deg).value, dec.to(u.deg).value, lonlat=True)

        sel_pix = hp.query_disc(nside_skymap, vecs, radius.to(u.rad).value, inclusive=True, fact=fact)
        
        #m,s = np.median(distmu[sel_pix]), np.median(distsigma[sel_pix])
        
        currcat = cat2[np.isin(np.array(cat2.PIX_ID_2048), sel_pix)][['Dist','Mstar']]
        #currcat = currcat[(currcat.Dist>=m-3*s) & (currcat.Dist<=m+3*s)]
        t
        masses3 = np.sum(currcat['Mstar']) * np.sum(prob2d[sel_pix])
        
        sigma_m_empty_nodist[h] = masses3
        prob2d[sel_pix] = 0.0
        print(h)

In [ ]:
d1 = pd.read_csv('full.csv')

names = d1['num'][304:]

for name in names:
    
    skym = ligo.skymap.io.fits.read_sky_map('farah/allsky/{}.fits'.format(name), distances=True)
    skymap = hp.ud_grade(skym[0][0], 512)
    
    print('\n\n\n##############################',name,'#################################\n\n\n')

    tiles = pd.read_csv('tiles_GIT_7.csv')
    ralist, declist = np.array(tiles['RA_Center'])*u.deg, np.array(tiles['DEC_Center'])*u.deg
    probs = get_probabilities(skymap, ralist, declist)
    tt = get_top_tiles(probs, 0.99)
    len(tt)

    distmu = hp.ud_grade(skym[0][1], 512)
    distsigma = hp.ud_grade(skym[0][2], 512)
    prob2d = hp.ud_grade(skym[0][0], 512)


    radius = 0.35*u.deg
    fact = 1
    nside_skymap = 512

    tile_area = np.pi * radius.to(u.deg).value ** 2
    pixel_area = hp.nside2pixarea(nside_skymap, degrees=True)

    sigma_m = np.zeros(len(tt))

    mpcd = np.arange(0,1001,20)


    sigma_m_empty_dist = np.zeros(len(tt))
    sigma_m_empty_nodist = np.zeros(len(tt))
    sigma_m_nonempty = np.zeros(len(tt))
    sigma_m_noprob = np.zeros(len(tt))
    sigma_m_noprob_filled = np.zeros(len(tt))
    
    #if len(tt)

    for h in range(len(tt)):
        ra = ralist[tt[h]]
        dec = declist[tt[h]]

        vecs = hp.ang2vec(ra.to(u.deg).value, dec.to(u.deg).value, lonlat=True)

        sel_pix = hp.query_disc(nside_skymap, vecs, radius.to(u.rad).value, inclusive=True, fact=fact)

        masses1 = np.zeros(len(sel_pix))
        masses2 = np.zeros(len(sel_pix))
        masses3 = np.zeros(len(sel_pix))
        masses4 = np.zeros(len(sel_pix))
        masses5 = np.zeros(len(sel_pix))

        mpcd = np.arange(0,1001,20)

        for j in range(len(sel_pix)):
            i = sel_pix[j]
            currcat = (cat2[cat2.PIX_ID_512 == i][['Dist','Mstar']])

            currcat.reset_index(drop=True, inplace=True)

            indices = np.arange(0,50,1)
            filled = []

            distfact = 1/distsigma[i] * np.exp( -(mpcd-distmu[i])**2 / (2*distsigma[i]**2) )

            for k in range(len(currcat)):
                ind = np.argmin(np.abs(new['distcen'] - currcat['Dist'][k]))
                filled.append(ind)

            unfilled = np.delete(indices, filled)
            masses1[j] = prob2d[i]*(np.sum(currcat['Mstar']*distfact[filled]) + np.sum(new['masstofill'][unfilled]*distfact[unfilled] / (new['num_empty_pixels'][unfilled])))

            masses2[j] = prob2d[i]*np.sum(currcat['Mstar']*distfact[filled])

            masses3[j] = prob2d[i]*np.sum(currcat['Mstar'])
            
            masses4[j] = np.sum(currcat['Mstar'])
            
            masses5[j] = masses4[j] + np.sum(new['masstofill'][unfilled] / (new['num_empty_pixels'][unfilled]))

        prob2d[sel_pix] = 0.0

        sigma_m_empty_dist[h] = np.sum(masses2)
        sigma_m_empty_nodist[h] = np.sum(masses3)
        sigma_m_nonempty[h] = np.sum(masses1)
        sigma_m_noprob[h] = np.sum(masses4)
        sigma_m_noprob_filled[h] = np.sum(masses5)
        print(h)
        
    df = pd.DataFrame({'Filled3D':sigma_m_nonempty, 'Unfilled3D':sigma_m_empty_dist, 'Unfilled2D':sigma_m_empty_nodist, 'NoProb':sigma_m_noprob, 'NoProbFilled':sigma_m_noprob_filled})
    df.to_csv('Orderings/Map_{}.csv'.format(name))

# MAIN CELL END. 

## NEXT FEW CELLS DO THE GLOBAL CALCULATION FOR EACH MAP - CAN LEAVE IT TILL ALL MASS-FILLING ORDERINGS HAVE BEEN RUN, AND I CAN DO IT AFTER NEXT WEEK


In [ ]:
d1

In [ ]:
latest = 0

d1 = pd.read_csv('full.csv')
ar = []
dis = []
a1, a2, a3, a4, a5, a6 = [],[],[],[], [],[]
ref = pd.read_csv('full2.csv')
nobs = 100

for n in d1['num'][latest:]:
    cs = pd.read_csv('Orderings/Map_{}.csv'.format(n))
    sigma_m_nonempty, sigma_m_empty_nodist, sigma_m_empty_dist, sigma_m_noprob = np.array(cs['Filled3D']), np.array(cs['Unfilled2D']), np.array(cs['Unfilled3D']), np.array(cs['NoProb'])
    fs = pd.read_csv('Orderings-2D-GIT/Map_{}.csv'.format(n))
    sigma_2dmp, monly = np.array(fs['Unfilled2D']), np.array(fs['MassOnly_Distcut'])
    #sigma_m_empty_nodist = np.array(pd.read_csv('Orderings-2D/Map_{}.csv'.format(n))['Unfilled2D'])
    
    order_dist = np.flip(np.argsort(sigma_m_empty_dist)[-nobs:])
    order_nodist = np.flip(np.argsort(sigma_m_empty_nodist)[-nobs:])
    order_all = np.flip(np.argsort(sigma_m_nonempty)[-nobs:])
    order_noprob = np.flip(np.argsort(sigma_m_noprob)[-nobs:])
    order_2dmp = np.flip(np.argsort(sigma_2dmp)[-nobs:])
    order_mdcut = np.flip(np.argsort(monly)[-nobs:])

    fullsum = np.sum(sigma_m_nonempty)
    a1.append(np.cumsum((sigma_m_nonempty)[order_nodist])[-1] / fullsum)
    a2.append(np.cumsum((sigma_m_nonempty)[order_dist])[-1] / fullsum)
    a3.append(np.cumsum((sigma_m_nonempty)[order_all])[-1] / fullsum)
    a4.append(np.cumsum((sigma_m_nonempty)[order_noprob])[-1] / fullsum)
    a5.append(np.cumsum((sigma_m_nonempty)[order_2dmp])[-1] / fullsum)
    a6.append(np.cumsum((sigma_m_nonempty)[order_mdcut])[-1] / fullsum)
    print(n)
    #print(k, a3[-1], a1[-1], a2[-1])
        
    ar.append(0)
    
    dis.append(0)
    
d1['DegArea'][latest:] = ref['DegArea']
d1['Mean_Dist'][latest:] = ref['Mean_Dist']
d1['P_3dcat'][latest:] = a2
d1['P_2dcat_dcut'][latest:] = a5
d1['P_2dcat'][latest:] = a1
d1['P_massfill'][latest:] = a3
d1['P_onlymass'][latest:] = a4
d1['P_massdcut'][latest:] = a6

In [ ]:
#d1 = pd.read_csv('BNS_full.csv')
p2d = []

for n in d1['num'][latest:]:
    file = pd.read_csv('Orderings/Map_{}.csv'.format(n))
    #print(len(file))
    p2 = np.sum(file.Filled3D[:nobs]) / np.sum(file.Filled3D[:])
    p2d.append(p2)
p2d    
d1['P_2d'][latest:] = p2d
d1

In [ ]:
d1.to_csv('Sept/full-GIT100.csv')

In [ ]:
np.where(d1.P_massfill < d1.P_2d)

In [ ]:
f = pd.read_csv('')

In [ ]:
d1.to_csv('BNS_full.csv')

## End of file writing